# 생성 모델을 위한 딥러닝

## 8.1 LSTM으로 텍스트 생성

### 8.1.4 글자 수준의 LSTM 텍스트 생성 모델 구현

In [1]:
import tensorflow.keras
import numpy as np

path = tensorflow.keras.utils.get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print ('말뭉치 크기:', len(text))

C:\Users\Roytravel\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Roytravel\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Roytravel\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Roytravel\Anaconda3\lib\site-packages\tensorboard\compat\tensorf

606208/600901 [==============================] - 1s 2us/step
말뭉치 크기: 600893


### 코드 8-3 글자 시퀀스 벡터화

In [3]:
maxlen = 60 # 60개의 글자로 된 시퀀스 추출
step = 3 # 세 글자씩 건너뛰면서 새로운 시퀀스 샘플링

sentences = list() #추출한 시퀀스 담을 리스트

next_chars = list() # 타깃(시퀀스 다음 글자)을 담을 리스트

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i:i+maxlen])
    next_chars.append(text[i+maxlen])

print('시퀀스 개수:', len(sentences))

chars = sorted(list(set(text))) # 말뭉치에서 고유한 글자를 담은 리스트
print('고유한 글자:', len(chars))
char_indices = dict((char, chars.index(char)) for char in chars) # chars 리스트에 있는 글자와 글자의 인덱스를 매핑한 딕셔너리

print('벡터화...') # 글자를 원-핫 인코딩하여 0과 1의 이진 배열로 변경
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
    

시퀀스 개수: 200278
고유한 글자: 58
벡터화...


네트워크 구성

### 코드 8-4 다음 글자를 예측하기 위한 단일 LSTM 모델

In [4]:
from tensorflow.keras import layers

model = tensorflow.keras.models.Sequential()
model.add(layers.LSTM(28, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

### 코드 8-5 모델 컴파일 설정

In [9]:
optimizer = tensorflow.keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

### 코드 8-6 모델의 예측이 주어졌을 때 새로운 글자를 샘플링하는 함수

In [10]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

### 코드 8-7 텍스트 생성 루프

In [13]:
import random
import sys

random.seed(42)
start_index = random.randint(0, len(text) - maxlen - 1)

for epoch in range(1,60):
    print('에포크', epoch)
    model.fit(x, y, batch_size=128, epochs=1)
    
    seed_text = text[start_index: start_index + maxlen]
    print('--- 시트 텍스트: "' + seed_text + '"')
    
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ 온도:', temperature)
        generated_text = seed_text
        sys.stdout.write(generated_text)
        
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.
                
            # 다음 글자 샘플링
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            
            generated_text += next_char
            generated_text = generated_text[1:]
            
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

에포크 1
1565/1565 [==============================] - 5s 3ms/step - loss: 1.8460
--- 시트 텍스트: "the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through for the manded and the soupt and mand of the self the most precindent and the self--inderion of the self and the and interes and the mantion of the self and the self--he and and the self and the self the mand interition of and the precindent of the self the self and the and the self--inderion of the self--and the world and and the and the most and the self and and the self in the self and the precin
------ 온도: 0.5
the slowly ascending ranks and classes, in which,
through for the religion and soulded in the good intend action one he experition intersion in the comperpersion the has as the gerenting somether sace of mein of an an and comption be to be the from and indicity, and an the germates the all present of relaction of the shere and the prectiand in the someth

hermarity, standeun=-evensique, to depafital, if i him is riys of gunife upon theheptouna
에포크 5
1565/1565 [==============================] - 5s 3ms/step - loss: 1.7542
--- 시트 텍스트: "the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through for the anconce the still and the proved and the present and the moraling the anconst the even the man and the man the more the man and the spirition of the any of the sting of the man and the strong the and the anconcess to the anconcess of the an the anconse and the strom the more the spirition of the most and the spirition to the more the man and the spirition of the man and the strong the strom 
------ 온도: 0.5
the slowly ascending ranks and classes, in which,
through for the manines and set it the fain the any and the spirition of and the does conteress senting the sarfords the desore comitions of pertopling and the spiction of the excection of the spremers of may dispari

is gont good", and requen gater--jusoinates-"""" in
"gording, inscion not bleaking so roble most guritions of
nacent thear--is poighe (or,
usk,
he latull,,
에포크 9
1565/1565 [==============================] - 5s 3ms/step - loss: 1.7283
--- 시트 텍스트: "the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through for the a predicies of the and the manding the suphould and a predicies and which the suphould the supertion of the many the and the manding the supertion the precerses the suphour the any and the any and the and the any and a such the manding of the suble and the spiriting the suphour the a precthion the and the suphour the supertion of the form-which the great the a such a precisely the a precisel
------ 온도: 0.5
the slowly ascending ranks and classes, in which,
through for the manding the philosophers, for the any such suble to a proogise the bor predarity and diffictic brought make to the all the greatity th

condacerluof townert laful, ovires wommant schesine, escer? dorpition, tnie, slat; foded do nupposs," this: perhorirais,
valomes.=--what popreste: not in the grait
tomes.--no philisifect brough
the in its
peratual sopeliey usshal gretcence rusiolly," it, to wh
에포크 13
1565/1565 [==============================] - 5s 3ms/step - loss: 1.7157
--- 시트 텍스트: "the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through for the feeling of the somet the still and a presest the recome the shought the philosophy of the spirit and and the supreble and the should and the precisely and the are a somet the spirit of the whoth and the a proves and sufficient and all the a proves and and and the every and the spirit and the shought and and a precisely something the still a somet the spirit and still a proves and a such and 
------ 온도: 0.5
the slowly ascending ranks and classes, in which,
through for the such of the germany somet sh

not schoupeishwions, then or irbiy attopice; but apoptsther, thenle has yaniss of in dobive. makes to as though whol hinkers to crightaght crasim to the
scary hassion; evarces cans-institress sand, is semprimed you--but he the
crovitist
or sires chrypantity
sepsecrom! hild se owin at
perhise, "gound chultal wi all utons,
ave
wes socess orfevel, the to adjususre.


1
inself-germortarred they 
에포크 17
1565/1565 [==============================] - 5s 3ms/step - loss: 1.7091
--- 시트 텍스트: "the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through for the of the still to and the and the spirit and and the to the and the still the restented and to and the whole and the and the strom the stroge and the a to the stand of the and the suphoust and the suphoust and the subth the and the and and the suphoust of the and to the suphoust to the many and the and the any and the suphoust and the destinct of the suphoust and the an

...

through forticul
sence
hut, by picvic"). the predateticly, and, ratiousted, of which reperiessary
who and "forechest, when
prity-offrated the pladin.,--nemilin: ;  xucs oh will
to fur
couned lite of
the enofock
accompation
of or
cosm sacom and. hich
due!

he ,n -it ori, and peader-add so i womanition a
wold
the layn is to known there, socul", the wept sified tness
of is: now bavem--iss thought of the refee 
에포크 41
1565/1565 [==============================] - 5s 3ms/step - loss: 1.6938
--- 시트 텍스트: "the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through for the moral a sentiment the still the moral the summay and supremation the spirit and the sentim and the still the man the sentiment and the good the developer in the man and the saince and the many and the man and with the suprement the man the suppose of a predicic a sentiment the strenge and and the still a sentiment the spirit, which the still the man is

through for is persislo teal. a greeble inabluch, thene: heors by inclon it fisefly i umpeedornces. a
somantly diffevises thmatided funts, soged the phirance.
of
toudice,,
for all wat nasoonness being geedes belelf pant prectinged evamilyzing ame, it
a lating is phop to
than that that lacinbidatiosmblemene upontradis. moralist unfours of
is winhts evil: jester
in of handy of apleles flovethed. to smastemagl
에포크 45
1565/1565 [==============================] - 5s 3ms/step - loss: 1.6916
--- 시트 텍스트: "the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through for the subtle of the should and subtle of the has the spirit, in the spirit and the such the world and the something and the spirit and the more the subject the spirit and the something and of the still the something the something and the subject that the spirit, the spelsion and a such something and and such that the senten and the something and the more the

through foree; himsech, onts. in teem ger ont
tourcians andness oifre;ted's, thaty lays under
it intorent calation herre or much torled which mesters. the mouss extyroncce shorr
soothelfult uponece and justis
of cunttizeands
reare.--and extered ercerelies stapth is corel coseves to the succest of bler--so lighlars, "call," ,amrest rest the
many from purtuence presents. dilit
of have the upin casre the teare
에포크 49
1565/1565 [==============================] - 5s 3ms/step - loss: 1.6922
--- 시트 텍스트: "the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through for the seen and the seen and the man and the acture and the seen and sense of the sense and the seen the seen and sense the more the seen and the seen and account and the seeken the man and and seen the such a desire the actount of the morality of the more and the actount the many the seen the seetises of the seen the stand of the stand of the stand of the ac

through foill
theid be kyote, ideanwimay youl mustog's,
brivelicm.--bupble thelwsh-the that old grands with traluty too acquirely, he poalitioned rely compinomat
one--byelf if ""modent,
delegaine--guin. theres, skited)-grepints" rimd higher. this much: aveaser--she inenit ipleal
forwles, and actiomtic-der a men estimethoswence, and lovios and
as
curierg.

1ly the opremilere which pritemm. thery kncy eloned 
에포크 53
1565/1565 [==============================] - 5s 3ms/step - loss: 1.6907
--- 시트 텍스트: "the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through for the man and the action of the self-the stand of the most the most the man and the man and the still the senten the self-the sense the stress and such the still and the sense the perhaps and the persons and the man and the spirit, and the many and the sense the still the sense the decealing to the experies the sense as the senten the perhaps the most the ma

through formed beiness"rawial -gil the mates iplightly shar
lofent,
his efforvere selations,, but from finifach, bedabs's desetre lives was enfinesdity iltemeurs to migingpisg an figiupist, evient but as "max are scoion;wensd the griemplize?! of the
ageing older whosel
discover: aruphisiond our ?  bersities
winh
.)
fore
rirdent
craedory andavadys, themeles, ought which thymowacully commabladed of spouned ag
에포크 57
1565/1565 [==============================] - 5s 3ms/step - loss: 1.6887
--- 시트 텍스트: "the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through for the stand of the soun and the soul and the precisele and the subject the soul the seem of the stand of the and the the more and the sense of the deman the spirit, the sound the such and the man the soun of the any some and the act the more and the soun, and the man the more and the soun of the soun and the sense the and the stand of the and the and the sou

In [14]:
model.save('./models/nietzsche_with_lstm.h5')